In [1]:
import pandas as pd

train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

# 資料處理
    MSSubClass: one-hot encoding 處理

In [2]:
train_df_drop = train_df.drop(['SalePrice'], axis=1)
datas = pd.concat([train_df_drop, test_df], axis=0)
datas = datas.drop(['Id'], axis=1)

In [3]:
# datas.shape

In [4]:
out_list = ['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'FireplaceQu']
train_copy = datas.drop(out_list, axis=1)

In [5]:
# nan = train_copy.isna().sum()
# nan[nan.values != 0].sort_values(ascending=False)

## 填入中間值

In [6]:
med = train_copy.median()
train_copy = train_copy.fillna(med)
# train_copy.isna().sum()

In [7]:
# nan = train_copy.isna().sum()
# nan[nan.values != 0].sort_values(ascending=False)

In [8]:
fill_most_list = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
                  'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                  'Electrical', 'KitchenQual', 'Functional', 
                  'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'SaleType']

## 填入 常用值

In [9]:
for s in fill_most_list:
    most = train_copy[s].value_counts().idxmax()
    train_copy[s] = train_copy[s].fillna(most)
# train_copy.isna().sum()

In [10]:
# nan = train_copy.isna().sum()
# nan[nan.values != 0].sort_values(ascending=False)

## TODO: 預處理資料

In [17]:
train_copy = pd.get_dummies(train_copy)
# train_copy = pd.get_dummies(train_copy, columns=["MSSubClass"])
# train_copy

In [18]:
nan = train_copy.isna().sum()
nan[nan.values != 0].sort_values(ascending=False)

Series([], dtype: int64)

In [19]:
# 取列: .iloc -> [第一筆資料, 第二筆資料, ..., 第 N 筆資料]
x_train = train_copy.iloc[: train_df.shape[0]]
y_train = train_df['SalePrice']
x_predict = train_copy.iloc[train_df.shape[0]: ]
predict_id = test_df['Id']

## TODO: RandomForestRegressor

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# clf = RandomForestClassifier()
# params = {
#     'n_estimators': range(20, 35),
#     'max_depth': range(6, 10)
# }
# grid = GridSearchCV(clf, params, cv=10, n_jobs=4)
# grid.fit(x_train, y_train)
# print('最佳參數: ', grid.best_params_)
# print('最佳分數 (10 次分數): ', grid.best_score_)

In [21]:
from sklearn.model_selection import cross_val_score

reg = RandomForestClassifier(n_estimators=75, max_depth=8)
scores = cross_val_score(reg,
                         x_train,
                         y_train,
                         cv=10,
                         n_jobs=4)

print('十次分數: ', scores)
print('平均分數: ', sum(scores) / len(scores))

e:\class\6_tibame\05_mldemo_01_v1903\venv\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


十次分數:  [0.01369863 0.02739726 0.01369863 0.00684932 0.01369863 0.00684932
 0.00684932 0.00684932 0.         0.00684932]
平均分數:  0.010273972602739725


In [22]:
reg.fit(x_train, y_train)

pre = reg.predict(x_predict)

result = pd.DataFrame({
    "Id": predict_id,
    "SalePrice": pre
})
result.to_csv('house_price_rf.csv', encoding='utf-8', index=False)
result

,Id,SalePrice
0,1461,140000
1,1462,151500
2,1463,185000
3,1464,189000
4,1465,180000
...,...,...
1454,2915,118000
1455,2916,82000
1456,2917,141000
1457,2918,147000
